<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/unit3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="In Colab öffnen"/></a>

# Einheit 3: Tiefes Q-Learning mit Atari-Spielen 👾 mit RL Baselines3 Zoo

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/thumbnail.jpg" alt="Unit 3 Thumbnail">

In diesem Notizbuch **trainieren Sie einen Deep Q-Learning-Agenten**, der Space Invaders spielt. Dazu verwenden Sie [RL Baselines3 Zoo] (https://github.com/DLR-RM/rl-baselines3-zoo), ein auf [Stable-Baselines3] (https://stable-baselines3.readthedocs.io/en/master/) basierendes Trainingsframework, das Skripte für das Training, die Auswertung von Agenten, die Abstimmung von Hyperparametern, die Darstellung von Ergebnissen und die Aufnahme von Videos bereitstellt.

Wir verwenden die [RL-Baselines-3 Zoo Integration, eine Vanilla-Version von Deep Q-Learning](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) ohne Erweiterungen wie Double-DQN, Dueling-DQN und Prioritized Experience Replay.

⬇️ Hier ein Beispiel dafür, was **Sie erreichen werden** ⬇️

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/ThomasSimonini/ppo-SpaceInvadersNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

### 🎮 Umgebungen:

- [SpacesInvadersNoFrameskip-v4](https://gymnasium.farama.org/environments/atari/space_invaders/)

Sie können den Unterschied zwischen den Space Invaders Versionen hier sehen 👉 https://gymnasium.farama.org/environments/atari/space_invaders/#variants

### 📚 RL-Library:

- [RL-Baselines3-Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

## Ziele dieses Notizbuchs 🏆
Am Ende des Heftes wirst du:
- In der Lage sein, tiefer zu verstehen **wie RL Baselines3 Zoo funktioniert**.
- In der Lage sein, **Ihren trainierten Agenten und den Code in den Hub** mit einer schönen Videowiedergabe und einem Bewertungsergebnis zu pushen 🔥.




## Dieses Notebook stammt aus dem Deep Reinforcement Learning Kurs.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

In diesem kostenlosen Kurs lernen Sie:

- 📖 Deep Reinforcement Learning in **Theorie und Praxis** studieren.
- 🧑‍💻 Lernen Sie, **berühmte Deep RL-Bibliotheken** wie Stable Baselines3, RL Baselines3 Zoo, CleanRL und Sample Factory 2.0 zu verwenden.
- 🤖 Trainieren Sie **Agenten in einzigartigen Umgebungen**.

Und mehr, siehe 📚 den Lehrplan 👉 https://simoninithomas.github.io/deep-rl-course

Vergessen Sie nicht, sich **<a href="http://eepurl.com/ic5ZUD">für den Kurs anzumelden</a>** (wir sammeln Ihre E-Mail, um Ihnen **die Links zu senden, wenn die einzelnen Einheiten veröffentlicht werden, und Sie über die Herausforderungen und Aktualisierungen zu informieren).**


Der beste Weg, um in Kontakt zu bleiben, ist, unserem Discord-Server beizutreten, um sich mit der Community und mit uns auszutauschen 👉🏻 https://discord.gg/ydHrjt3WP5

## Voraussetzungen 🏗️
Bevor Sie sich mit dem Notebook beschäftigen, müssen Sie:

🔲 📚 **[Deep Q-Learning durch Lesen von Einheit 3](https://huggingface.co/deep-rl-course/unit3/introduction)** 🤗

Wir versuchen ständig, unsere Anleitungen zu verbessern. **Wenn Sie also Probleme in diesem Notizbuch** finden, öffnen Sie bitte [ein Problem im Github Repo](https://github.com/huggingface/deep-rl-class/issues).

# Trainieren wir einen Deep-Q-Learning-Agenten, der Atari' Space Invaders 👾 spielt, und laden wir ihn in den Hub hoch.

Wir empfehlen den Schülerinnen und Schülern dringend, **Google Colab für die praktischen Übungen zu verwenden, anstatt sie auf ihren Computern auszuführen**.

Durch die Verwendung von Google Colab können **Sie sich auf das Lernen und Experimentieren konzentrieren, ohne sich um die technischen Aspekte der Einrichtung Ihrer Umgebungen zu kümmern**.

Um diese praktische Übung für den Zertifizierungsprozess zu validieren, müssen Sie Ihr trainiertes Modell an den Hub senden und **ein Ergebnis von >= 200** erhalten.

Um Ihr Ergebnis zu ermitteln, gehen Sie zum Leaderboard und suchen Sie Ihr Modell, **das Ergebnis = mean_reward - std of reward**

Weitere Informationen über den Zertifizierungsprozess finden Sie in diesem Abschnitt 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Ein Ratschlag 💡
Es ist besser, dieses Colab in einer Kopie auf Ihrem Google Drive auszuführen, so dass Sie **bei Zeitüberschreitungen** immer noch das gespeicherte Notizbuch auf Ihrem Google Drive haben und nicht alles von Grund auf neu ausfüllen müssen.

Dazu kannst du entweder "Strg + S" oder "Datei > Kopie in Google Drive speichern" verwenden.

Außerdem werden wir es **90 Minuten lang mit 1M Zeitschritten** trainieren. Die Eingabe von `!nvidia-smi` verrät Ihnen, welche GPU Sie verwenden.

Wenn Sie mehr als 10 Millionen Schritte trainieren wollen, wird dies etwa 9 Stunden dauern, was möglicherweise zu einer Zeitüberschreitung von Colab führt. In diesem Fall empfehle ich, das Programm auf Ihrem lokalen Computer (oder irgendwo anders) auszuführen. Klicken Sie einfach auf: Datei>Download".

## Die GPU einstellen 💪.
- Um **das Training des Agenten zu beschleunigen, werden wir einen Grafikprozessor** verwenden. Gehen Sie dazu auf "Runtime > Change Runtime type".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Schritt 1">

- Hardware-Beschleuniger > GPU".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Schritt 2">

# RL-Baselines3 Zoo und seine Abhängigkeiten installieren 📚.

Wenn Sie "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed" sehen, **das ist normal und kein kritischer Fehler**, gibt es einen Versionskonflikt. Aber die Pakete, die wir brauchen, sind installiert.

In [ ]:
# For now we install this update of RL-Baselines3 Zoo
!pip install git+https://github.com/DLR-RM/rl-baselines3-zoo@update/hf

WENN UND NUR WENN DIE OBIGE VERSION NICHT MEHR EXISTIERT. ENTKOMMENTIEREN UND INSTALLIEREN SIE DIE UNTEN STEHENDE VERSION

In [ ]:
#!pip install rl_zoo3==2.0.0a9

In [ ]:
!apt-get install swig cmake ffmpeg

Um Atari-Spiele in Gymnasium verwenden zu können, müssen wir das Atari-Paket installieren. Und accept-rom-license, um die Rom-Dateien (Spiele-Dateien) herunterzuladen.

In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

## Erstellen einer virtuellen Anzeige 🔽.

Während der Arbeit mit dem Notebook müssen wir ein Wiederholungsvideo erstellen. Dazu benötigen wir mit colab **einen virtuellen Bildschirm, um die Umgebung zu rendern** (und somit die Bilder aufzunehmen).

Daher wird die folgende Zelle die Librairies installieren und einen virtuellen Bildschirm erstellen und starten 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Trainiere unseren Deep Q-Learning Agent, um Space Invaders zu spielen 👾

Um einen Agenten mit RL-Baselines3-Zoo zu trainieren, müssen wir nur zwei Dinge tun:

1. Erstellen Sie eine Hyperparameter-Konfigurationsdatei, die unsere Trainings-Hyperparameter mit dem Namen "dqn.yml" enthalten wird.

Dies ist ein Beispiel für eine Vorlage:

```
SpaceInvadersNoFrameskip-v4:
  env_wrapper:
    - stable_baselines3.common.atari_wrappers.AtariWrapper
  frame_stack: 4
  Richtlinie: 'CnnPolicy'
  n_timesteps: !!float 1e6
  puffer_größe: 100000
  lern_rate: !!float 1e-4
  batch_size: 32
  lern_starts: 100000
  Ziel_aktualisieren_Intervall: 1000
  train_freq: 4
  gradient_steps: 1
  Erkundung_Anteil: 0.1
  exploration_final_eps: 0.01
  # Wenn True, müssen Sie handle_timeout_termination deaktivieren.
  # in den replay_buffer_kwargs
  optimize_memory_usage: False
```

Hier sehen wir das:
- Wir verwenden den "Atari Wrapper", der die Eingabe vorverarbeitet (Frame-Reduktion, Graustufen, 4 Frames stapeln)
- Wir verwenden `CnnPolicy`, da wir Faltungsschichten zur Verarbeitung der Frames verwenden
- Wir trainieren es für 10 Millionen `n_Zeitschritte`
- Die Größe des Speichers (Experience Replay) beträgt 100000, d.h. die Anzahl der Erfahrungsschritte, die Sie gespeichert haben, um Ihren Agenten erneut zu trainieren.

💡 Mein Rat ist, **die Trainingszeitschritte auf 1M zu reduzieren,** was auf einem P100 etwa 90 Minuten dauern wird. !nvidia-smi" wird Ihnen sagen, welche GPU Sie verwenden. Bei 10 Millionen Schritten wird dies etwa 9 Stunden dauern, was wahrscheinlich zu einer Zeitüberschreitung von Colab führen könnte. Ich empfehle, dies auf Ihrem lokalen Computer (oder irgendwo anders) auszuführen. Klicken Sie einfach auf: Datei>Download".

Was die Optimierung der Hyperparameter angeht, so rate ich, sich auf diese 3 Hyperparameter zu konzentrieren:
- `Lernrate`
- Puffergröße (Erfahrungsspeichergröße)`
- `Stapelgröße`

Als gute Praxis sollten Sie **die Dokumentation lesen, um zu verstehen, was jeder Hyperparameter bewirkt**: https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters



2. Wir starten das Training und speichern die Modelle im Ordner "Logs" 📁.

- Wir definieren den Algorithmus nach `--algo`, speichern das Modell nach `-f` und die Konfiguration der Hyperparameter nach `-c`.

In [ ]:
!python -m rl_zoo3.train --algo ________ --env SpaceInvadersNoFrameskip-v4  -f _________  -c _________

#### Lösung

In [ ]:
!python -m rl_zoo3.train --algo dqn  --env SpaceInvadersNoFrameskip-v4 -f logs/ -c dqn.yml

## Lass uns unseren Agenten auswerten 👀
- RL-Baselines3-Zoo bietet `enjoy.py`, ein Python-Skript zur Auswertung unseres Agenten. In den meisten RL-Bibliotheken nennen wir das Auswertungsskript `enjoy.py`.
- Lassen wir es für 5000 Zeitschritte auswerten 🔥.

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps _________  --folder logs/ 

#### Lösung

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps 5000  --folder logs/

## Veröffentliche unser trainiertes Modell auf dem Hub 🚀
Da wir nun gesehen haben, dass wir nach dem Training gute Ergebnisse erzielt haben, können wir unser trainiertes Modell mit einer Zeile Code auf dem Hub 🤗 veröffentlichen.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/space-invaders-model.gif" alt="Space Invaders model">

Mit `rl_zoo3.push_to_hub` **werten Sie aus, zeichnen ein Replay auf, generieren eine Modellkarte Ihres Agenten und schieben sie zum Hub**.

Auf diese Weise:
- Sie können **unsere Arbeit vorführen** 🔥.
- Sie können **Ihren Agenten beim Spielen visualisieren** 👀
- Du kannst **einen Agenten mit der Community teilen, den andere benutzen können** 💾
- Sie können **auf eine Bestenliste 🏆 zugreifen, um zu sehen, wie gut Ihr Agent im Vergleich zu Ihren Klassenkameraden abschneidet** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

Um Ihr Modell mit der Gemeinschaft teilen zu können, sind drei weitere Schritte erforderlich:

1️⃣ (Falls noch nicht geschehen) Erstellen Sie ein Konto für HF ➡ https://huggingface.co/join

2️⃣ Melde dich an und speichere dann dein Authentifizierungs-Token von der Hugging Face Website.
- Erstellen Sie ein neues Token (https://huggingface.co/settings/tokens) **mit Schreibrolle**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF-Token erstellen">

- Kopieren Sie das Token
- Führen Sie die Zelle unten aus und fügen Sie das Token ein

In [ ]:
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
notebook_login()
!git config --global credential.helper store

Wenn Sie kein Google Colab oder ein Jupyter Notebook verwenden möchten, müssen Sie stattdessen diesen Befehl verwenden: `huggingface-cli login`

3️⃣ Wir sind jetzt bereit, unseren geschulten Agenten zum 🤗 Hub 🔥 zu bringen.

Führen wir die Datei push_to_hub.py aus, um unseren trainierten Agenten in den Hub hochzuladen.

`-Repo-Name`: Der Name des Repo

`-orga`: Ihr Umarmungsgesicht-Benutzername

`-f`: Wo sich der Ordner des trainierten Modells befindet (in unserem Fall `logs`)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/select-id.png" alt="Id auswählen">

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name _____________________ -orga _____________________ -f logs/

#### Lösung

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name dqn-SpaceInvadersNoFrameskip-v4  -orga ThomasSimonini  -f logs/

###.

Herzlichen Glückwunsch 🥳 Sie haben gerade Ihren ersten Deep Q-Learning-Agenten mit RL-Baselines-3 Zoo trainiert und hochgeladen. Das obige Skript sollte einen Link zu einem Modell-Repository wie https://huggingface.co/ThomasSimonini/dqn-SpaceInvadersNoFrameskip-v4 angezeigt haben. Wenn Sie zu diesem Link gehen, können Sie:

- Eine **Video-Vorschau Ihres Agenten** auf der rechten Seite sehen.
- Klicken Sie auf "Dateien und Versionen", um alle Dateien im Repository zu sehen.
- Klicken Sie auf "Use in stable-baselines3", um ein Codeschnipsel zu erhalten, das zeigt, wie man das Modell lädt.
- Eine Modellkarte (Datei `README.md`), die eine Beschreibung des Modells und der verwendeten Hyperparameter enthält.

Unter der Haube verwendet der Hub git-basierte Repositories (keine Sorge, wenn Sie nicht wissen, was git ist), was bedeutet, dass Sie das Modell mit neuen Versionen aktualisieren können, wenn Sie experimentieren und Ihren Agenten verbessern.

**Vergleiche die Ergebnisse deiner Agenten mit denen deiner Klassenkameraden** mit Hilfe des [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) 🏆

## Laden Sie ein leistungsstarkes trainiertes Modell 🔥.
- Das Stable-Baselines3-Team hat **mehr als 150 trainierte Deep Reinforcement Learning-Agenten auf den Hub** hochgeladen.

Sie können sie hier finden: 👉 https://huggingface.co/sb3

Einige Beispiele:
- Asteroiden: https://huggingface.co/sb3/dqn-AsteroidsNoFrameskip-v4
- Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4
- Breakout: https://huggingface.co/sb3/dqn-BreakoutNoFrameskip-v4
- Road Runner: https://huggingface.co/sb3/dqn-RoadRunnerNoFrameskip-v4

Laden wir einen Agenten, der Beam Rider spielt: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

1. Wir laden das Modell mit `rl_zoo3.load_from_hub` herunter und legen es in einem neuen Ordner ab, den wir `rl_trained` nennen können

In [ ]:
# Download model and save it into the logs/ folder
!python -m rl_zoo3.load_from_hub --algo dqn --env BeamRiderNoFrameskip-v4 -orga sb3 -f rl_trained/

2. Lassen Sie uns für 5000 Zeitschritte auswerten, ob

In [ ]:
!python -m rl_zoo3.enjoy --algo dqn --env BeamRiderNoFrameskip-v4 -n 5000  -f rl_trained/ --no-render

Warum nicht versuchen, Ihre eigenen **Deep Q-Learning Agent spielen BeamRiderNoFrameskip-v4 zu trainieren? 🏆.**

Wenn Sie versuchen wollen, überprüfen Sie https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4#hyperparameters **in der Modellkarte, haben Sie die Hyperparameter des trainierten Agenten.**

Aber die Suche nach Hyperparametern kann eine schwierige Aufgabe sein. Glücklicherweise werden wir in der nächsten Einheit sehen, wie wir **Optuna für die Optimierung der Hyperparameter 🔥 verwenden können.**


## Einige zusätzliche Herausforderungen 🏆
Die beste Art zu lernen **ist, Dinge selbst auszuprobieren**!

In der [Rangliste] (https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) findest du deine Agenten. Kannst du es an die Spitze schaffen?

Hier ist eine Liste von Umgebungen, mit denen du deinen Agenten trainieren kannst:
- BeamRiderNoFrameskip-v4
- BreakoutKeinFrameskip-v4
- EnduroKeinFrameskip-v4
- PongNoFrameskip-v4

Außerdem **wenn Sie lernen wollen, Deep Q-Learning selbst zu implementieren**, sollten Sie sich unbedingt die CleanRL-Implementierung ansehen: https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/dqn_atari.py

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/atari-envs.gif" alt="Umgebungen"/>

________________________________________________________________________
Herzlichen Glückwunsch zum Abschluss dieses Kapitels!

Wenn du dich immer noch verwirrt fühlst mit all diesen Elementen...das ist völlig normal! **So ging es mir und allen anderen, die RL studiert haben.

Nimm dir Zeit, um den Stoff wirklich zu **verstehen, bevor du weitermachst und die zusätzlichen Herausforderungen ausprobierst**. Es ist wichtig, diese Elemente zu beherrschen und eine solide Grundlage zu haben.

In der nächsten Einheit **werden wir etwas über [Optuna](https://optuna.org/)** lernen. Eine der wichtigsten Aufgaben beim Deep Reinforcement Learning ist es, einen guten Satz von Trainingshyperparametern zu finden. Und Optuna ist eine Bibliothek, die Ihnen hilft, die Suche zu automatisieren.






### Dies ist ein Kurs, der mit Ihnen aufgebaut wurde 👷🏿‍♀️

Schließlich wollen wir den Kurs mit Hilfe Ihres Feedbacks verbessern und aktualisieren. Wenn Sie welche haben, füllen Sie bitte dieses Formular aus 👉 https://forms.gle/3HgA7bEHwAmmLfwh9

Wir versuchen ständig, unsere Tutorials zu verbessern. **Wenn Sie also Probleme in diesem Notizbuch finden**, öffnen Sie bitte [einen Fehler im Github Repo] (https://github.com/huggingface/deep-rl-class/issues).

Wir sehen uns bei Bonuseinheit 2! 🔥

### Keep Learning, Stay Awesome 🤗